In [5]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen, Request
from user_agent import generate_user_agent
import re
from urllib.parse import urljoin
from tqdm import tqdm
import time
import mysql.connector
from tqdm.notebook import tqdm

### 이전코드 빽업

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen, Request
from user_agent import generate_user_agent
import re
from urllib.parse import urljoin
from tqdm import tqdm
import time


url = "https://www.chicagomag.com/chicago-magazine/november-2012/best-sandwiches-chicago/"

req = Request(url=url, headers={'User-Agent':generate_user_agent()})
page = urlopen(req)
soup = BeautifulSoup(page, "html.parser")

prefix = "https://www.chicagomag.com"

rank_list = []
menu_list = []
cafe_list = []
link_list = []

for item in soup.find_all("div", "sammy"):
    rank_list.append(item.find(class_="sammyRank").string)

    text_list = re.split(('\n'), item.find("a").get_text())
    menu_list.append(text_list[0])
    cafe_list.append(text_list[1])

    link_list.append(urljoin(prefix, item.find("a")["href"]))

data = {'Rank':rank_list, 'Menu':menu_list, 'Cafe':cafe_list, 'URL':link_list}
df = pd.DataFrame(data, columns=['Rank', 'Cafe', 'Menu', 'URL'])

price_list = []
address_list = []

for idx, row in tqdm(df.iterrows(), total=len(df.index)):
    req = Request(url=row['URL'], headers={'User-Agent':generate_user_agent()})
    page = urlopen(req)
    soup = BeautifulSoup(page, "html.parser")

    text = re.split('.,', soup.find('p', 'addy').get_text())[0]
    price = re.search('\$\d+\.(\d+)?', text).group()
    address = text[len(price) + 1:]

    price_list.append(price)
    address_list.append(address)

    time.sleep(1)

print(rank_list)
print(menu_list)
print(cafe_list)
print(link_list)
print(price_list)
print(address_list)


100%|██████████| 50/50 [01:56<00:00,  2.33s/it]

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50']
['BLT', 'Fried Bologna', 'Woodland Mushroom', 'Roast Beef', 'PB&L', 'Belgian Chicken Curry Salad', 'Lobster Roll', 'Smoked Salmon Salad', 'Atomica Cemitas', 'Grilled Laughing Bird Shrimp and Fried Po’ Boy', 'Ham and Raclette Panino', 'Breaded Steak', 'The Hawkeye', 'Chicken Dip', 'Wild Boar Sloppy Joe', 'Meatball Sub', 'Corned Beef', 'Turkey Club', 'Falafel', 'Crab Cake', 'Chicken Schnitzel', 'Shawarma', 'Toasted Pimiento Cheese', 'Vegetarian Panino', 'Cali Chèvre', 'Pastrami', 'The Fredo', 'Smoked Ham', 'Jibarito', 'Shaved Prime Rib', 'Serrano Ham and Manchego Cheese', 'Tuna Salad', 'Paramount Reuben', 'The Istanbul', 'B.A.D.', 'Duck Confit and Mozzarella', 'Croque Monsieur', 'Green Garbanzo', 'The

### 전체 로직 테스트

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen, Request
from user_agent import generate_user_agent
import re
from urllib.parse import urljoin
from tqdm import tqdm
import time


url = "https://www.chicagomag.com/chicago-magazine/november-2012/best-sandwiches-chicago/"

req = Request(url=url, headers={'User-Agent':generate_user_agent()})
page = urlopen(req)
soup = BeautifulSoup(page, "html.parser")

prefix = "https://www.chicagomag.com"


for idx, item in tqdm(enumerate(soup.find_all("div", "sammy"))):
    text = re.split(('\n'), item.find("a").get_text())
    menu = text[0]
    cafe = text[1]

    link = urljoin(prefix, item.find("a")["href"])

    req2 = Request(url=link, headers={'User-Agent':generate_user_agent()})
    page2 = urlopen(req2)
    soup2 = BeautifulSoup(page2, "html.parser")

    text2 = re.split('.,', soup2.find('p', 'addy').get_text())[0]
    price = re.search('\$\d+\.(\d+)?', text2).group()
    address = text2[len(price) + 1:]

    time.sleep(1)

0it [00:00, ?it/s]

1 BLT Old Oak Tap https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/ $10.  2109 W. Chicago Ave


1it [00:02,  2.44s/it]

2 Fried Bologna Au Cheval https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/ $9.  800 W. Randolph St


2it [00:04,  2.31s/it]

...

48it [01:52,  2.38s/it]

49 Le Végétarien Toni Patisserie https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Toni-Patisserie-Le-Vegetarien/ $8.75 . 65 E. Washington St


49it [01:54,  2.39s/it]

50 The Gatsby Phoebe’s Bakery https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Phoebes-Bakery-The-Gatsby/ $6.85 . 3351 N. Broadwa


50it [01:57,  2.35s/it]


### 샌드위치 테이블 생성

In [9]:
import mysql.connector

# MySQL 데이터베이스에 연결
conn = mysql.connector.connect(
    host = "studydb.clocgy680wga.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "ajferhcz135",
    database = "amrbase"
)
cursor = conn.cursor()

# 테이블 생성 SQL 쿼리 작성
create_sandwich_table_query = '''
CREATE TABLE IF NOT EXISTS sandwich (
    ranking INT,
    cafe VARCHAR(32),
    menu VARCHAR(64),
    price FLOAT,
    address VARCHAR(64)
)
'''

cursor.execute(create_sandwich_table_query) # 테이블 생성 쿼리 실행
conn.commit() # 변경 사항 저장

# 연결 종료
cursor.close()
conn.close()

### 최종

In [10]:
conn = mysql.connector.connect(
    host="studydb.clocgy680wga.ap-northeast-2.rds.amazonaws.com",
    port=3306,
    user="root",
    password="ajferhcz135",
    database="amrbase"
)
cursor = conn.cursor()

In [11]:
url = "https://www.chicagomag.com/chicago-magazine/november-2012/best-sandwiches-chicago/"

req = Request(url=url, headers={'User-Agent':generate_user_agent()})
page = urlopen(req)
soup = BeautifulSoup(page, "html.parser")

prefix = "https://www.chicagomag.com"

items = soup.find_all("div", "sammy")
for idx, item in tqdm(enumerate(items), total=len(items)):
    text = re.split(('\n'), item.find("a").get_text())
    cafe = text[0]
    menu = text[1]

    link = urljoin(prefix, item.find("a")["href"])

    req2 = Request(url=link, headers={'User-Agent':generate_user_agent()})
    page2 = urlopen(req2)
    soup2 = BeautifulSoup(page2, "html.parser")

    text2 = re.split('.,', soup2.find('p', 'addy').get_text())[0]
    price = re.search('\$\d+\.(\d+)?', text2).group()
    address = text2[len(price) + 1:]

    # 주소 앞의 점 제거
    if address.startswith('.'):
        address = address[1:].strip()

    # 데이터베이스에 데이터 삽입
    insert_query = '''
    INSERT INTO sandwich (ranking, cafe, menu, price, address)
    VALUES (%s, %s, %s, %s, %s)
    '''
    cursor.execute(insert_query, (idx+1, cafe, menu, float(price.replace('$', '')), address))
    conn.commit()

    time.sleep(1)

cursor.close()
conn.close()

  0%|          | 0/50 [00:00<?, ?it/s]